<h1>ETL</h1>

In [1]:
#Import modules
import numpy as np
import pandas as pd

<h2>Platforms datasets</h2>

In [ ]:
#Load the datasets
amazon = pd.read_csv('datasets/amazon_prime_titles.csv')
disney = pd.read_csv('datasets/disney_plus_titles.csv')
hulu = pd.read_csv('datasets/hulu_titles.csv')
netflix = pd.read_csv('datasets/netflix_titles.csv')

In [ ]:
#Add new id column with the first letter of each platform union show_id
amazon.insert(0, 'id', 'a' + amazon['show_id'].astype('string'))
disney.insert(0, 'id', 'd' + disney['show_id'].astype('string'))
hulu.insert(0, 'id', 'h' + hulu['show_id'].astype('string'))
netflix.insert(0, 'id', 'n' + netflix['show_id'].astype('string'))


In [6]:
#Fill with 'G' in empty rating column places
amazon['rating'] = amazon['rating'].fillna('G')
disney['rating'] = disney['rating'].fillna('G')
hulu['rating'] = hulu['rating'].fillna('G')
netflix['rating'] = netflix['rating'].fillna('G')


In [7]:
#Transform dates in format yyy-mm-dd
amazon['date_added'] = pd.to_datetime(amazon['date_added'])
disney['date_added'] = pd.to_datetime(disney['date_added'])
hulu['date_added'] = pd.to_datetime(hulu['date_added'])
netflix['date_added'] = pd.to_datetime(netflix['date_added'])

In [8]:
#Divide the column duration in duration_int and duration_type
amazon[['duration_int', 'duration_type']] = amazon.duration.str.split(' ', expand=True)
disney[['duration_int', 'duration_type']] = disney.duration.str.split(' ', expand=True)
hulu[['duration_int', 'duration_type']] = hulu.duration.str.split(' ', expand=True)
netflix[['duration_int', 'duration_type']] = netflix.duration.str.split(' ', expand=True)

In [9]:
#Change the text in lower case
listcolum = ['type','title','director','cast','country','listed_in','description','duration_type']
for item in listcolum:
    amazon[item] = amazon[item].str.lower()
for item in listcolum:
    disney[item] = disney[item].str.lower()
for item in listcolum:
    hulu[item] = hulu[item].astype('string') 
for item in listcolum:
    hulu[item] = hulu[item].str.lower()  
for item in listcolum:
    netflix[item] = netflix[item].str.lower()      

In [10]:
#Export csv
amazon.to_parquet('./transformed_datasets/amazon.parquet')
disney.to_parquet('./transformed_datasets/disney.parquet')
hulu.to_parquet('./transformed_datasets/hulu.parquet')
netflix.to_parquet('./transformed_datasets/netflix.parquet')

<h2>Categories dataset</h2>

In [5]:
#load parquet files
amazon = pd.read_parquet('datasets/amazon.parquet')
disney = pd.read_parquet('datasets/disney.parquet')
hulu = pd.read_parquet('datasets/hulu.parquet')
netflix = pd.read_parquet('datasets/netflix.parquet') 

In [14]:
#Extract the genres from multivalue column 
def categories_table():
    series = pd.Series([])
    for platform in [amazon, disney, hulu, netflix]:
        categories = platform['listed_in'].str.split(',', expand=True)
        categories = categories.stack()
        categories = categories.str.replace(' ', '')
        series = pd.concat([series, categories], ignore_index=True)
    y = np.sort(series.unique())
    return y

In [17]:
#use the function
cat = categories_table()
cat

array(['action', 'action&adventure', 'action-adventure', 'adultanimation',
       'adventure', 'andculture', 'animals&nature', 'animation', 'anime',
       'animefeatures', 'animeseries', 'anthology', 'arthouse', 'arts',
       'biographical', 'blackstories', 'britishtvshows', 'buddy',
       'cartoons', 'children&familymovies', 'classic&culttv',
       'classicmovies', 'classics', 'comedies', 'comedy', 'comingofage',
       'concertfilm', 'cooking&food', 'crime', 'crimetvshows',
       'cultmovies', 'dance', 'disaster', 'documentaries', 'documentary',
       'docuseries', 'drama', 'dramas', 'entertainment',
       'faith&spirituality', 'faithandspirituality', 'family', 'fantasy',
       'fitness', 'gameshow/competition', 'gameshows', 'health&wellness',
       'historical', 'history', 'horror', 'horrormovies',
       'independentmovies', 'international', 'internationalmovies',
       'internationaltvshows', 'kids', "kids'tv", 'koreantvshows',
       'latenight', 'latino', 'lgbtq', 'lgb

In [18]:
#creating a new dataframe with id and genres
categories = pd.DataFrame({"id_gen":[x for x in range(0,len(cat))], "genres": cat})

In [21]:
categories.head()

,id_gen,genres
0,0,action
1,1,action&adventure
2,2,action-adventure
3,3,adultanimation
4,4,adventure


In [22]:
#save the table into parquet file
categories.to_parquet("datasets/categories.parquet", index=False)

<h2>Platform_categories dataset</h2>

In [ ]:
#load parquet files
amazon = pd.read_parquet('datasets/amazon.parquet')
disney = pd.read_parquet('datasets/disney.parquet')
hulu = pd.read_parquet('datasets/hulu.parquet')
netflix = pd.read_parquet('datasets/netflix.parquet')

In [3]:
df = pd.read_parquet('datasets/categories.parquet')
df.head()

,id_gen,genres
0,0,action
1,1,action&adventure
2,2,action-adventure
3,3,adultanimation
4,4,adventure


In [58]:
#mapping id from platforms with genres in categories
platform_categories = pd.DataFrame(columns=["id","genres"])
platforms = [amazon, disney, hulu, netflix]
for plat in platforms: 
    for i in df['genres'].array:
        datafr = pd.DataFrame({"id":plat['id'].where(plat['listed_in'].str.contains(i)).dropna(), "genres": i})
        platform_categories = pd.concat([platform_categories, datafr])

In [60]:
print(platform_categories.shape)
platform_categories.head()

(53775, 2)


,id,genres
2,as3,action
8,as9,action
21,as22,action
29,as30,action
52,as53,action


In [61]:
#save into parquet file
platform_categories.to_parquet('datasets/platforms_categories.parquet', index=False)

<h2>Unified ratings dataset</h2>

In [2]:
#Load data
rating1 = pd.read_csv('datasets/ratings/1.csv')
rating2 = pd.read_csv('datasets/ratings/2.csv')
rating3 = pd.read_csv('datasets/ratings/3.csv')
rating4 = pd.read_csv('datasets/ratings/4.csv')
rating5 = pd.read_csv('datasets/ratings/5.csv')
rating6 = pd.read_csv('datasets/ratings/6.csv')
rating7 = pd.read_csv('datasets/ratings/7.csv')
rating8 = pd.read_csv('datasets/ratings/8.csv')


In [3]:
#concatenate dataframes
unified = pd.concat([rating1, rating2, rating3, rating4, rating5, rating6, rating7, rating8])

In [4]:
#change timestamp to datetime
unified['timestamp'] = pd.to_datetime(unified['timestamp'], unit='s')

In [6]:
unified.head()

,userId,rating,timestamp,movieId
0,1,1.0,2015-03-09 22:52:09,as680
1,1,4.5,2015-03-09 23:07:15,ns2186
2,1,5.0,2015-03-09 22:52:03,hs2381
3,1,5.0,2015-03-09 22:52:26,ns3663
4,1,5.0,2015-03-09 22:52:36,as9500


In [8]:
#Export in parquet format for memory issues 
unified.to_parquet('datasets/ratings_unified.parquet')